In [26]:
import json
import os
from datetime import datetime
from appdirs import user_data_dir


timestamp = datetime.now().strftime("%Y%m%d-%H-%M-%S")
duration_h = 2
algo = 'ori'

use_large = True
add_or_remove = True

app_path = f"/Users/dzakyzf/lain/iotlab/openwsniotlab/openwsn-fw/build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog"
site = "strasbourg"
energy_profile = "energym3"

small_nodes = [1,2,19,20,29,30,47,48,53,54] # 10
small_nodes_root = 29
small_near_root = 30
small_stop_nodes = [47,20]

large_nodes = small_nodes + [5,6,21,22,33,34,49,50,57,58] + [9,10,23,24,37,38,51,52,61,62] # 30
large_nodes_root = 33
large_near_root = 34
large_stop_nodes = [47,20] + [22,49] + [24,51]


if use_large:
    all_nodes = "+".join([str(x) for x in large_nodes])
    if add_or_remove:
        temp = [x for x in large_nodes if x not in large_stop_nodes]
        nodes = "+".join([str(x) for x in temp])
    else:
        nodes = all_nodes
    root = large_nodes_root
    near_root = large_near_root
    stop_nodes = "+".join([str(x) for x in large_stop_nodes])
else:
    all_nodes = "+".join([str(x) for x in small_nodes])
    if add_or_remove:
        temp = [x for x in small_nodes if x not in small_stop_nodes]
        nodes = "+".join([str(x) for x in temp])
    else:
        nodes = all_nodes
    root = small_nodes_root
    near_root = small_near_root
    stop_nodes = "+".join([str(x) for x in small_stop_nodes])

print(all_nodes)
print(nodes)
print(stop_nodes)

1+2+19+20+29+30+47+48+53+54+5+6+21+22+33+34+49+50+57+58+9+10+23+24+37+38+51+52+61+62
1+2+19+29+30+48+53+54+5+6+21+33+34+50+57+58+9+10+23+37+38+52+61+62
47+20+22+49+24+51


In [21]:
file_name = 'exp_log'
def get_log_path(file_name="x", format="txt", folder_only=False):
    if folder_only:
        return user_data_dir('openvisualizer')
    else:
        return os.path.join(user_data_dir('openvisualizer'), file_name+"."+format)

def run_command(cmd):
    print(cmd)
    val = !{cmd}
    val = [str(i).strip() for i in val]
    result = json.loads("".join(val))
    return result

### Submit experimet

In [18]:
# Submit experiment
cmd = f"iotlab-experiment submit -n test_{timestamp} -d {int(duration_h*60)} -l {site},m3,{all_nodes},,{energy_profile}"
print(cmd)

iotlab-experiment submit -n test_20230111-03-18-16 -d 120 -l strasbourg,m3,1+2+19+20+29+30+47+48+53+54+5+6+21+22+33+34+49+50+57+58+9+10+23+24+37+38+51+52+61+62,,energym3


In [4]:
result = run_command(cmd)
exp_id = result["id"]
exp_id

352894

In [5]:
!iotlab-experiment wait

Waiting that experiment 352894 gets in state Running
"Running"


In [19]:
stop_nodes

'47+20+22+49+24+51'

In [6]:
cmd = f'iotlab-node --flash {app_path} -l {site},m3,{all_nodes}'
run_command(cmd)

cmd = f'iotlab-node --reset -l {site},m3,{all_nodes}'
run_command(cmd)

['m3-1.strasbourg.iot-lab.info',
 'm3-2.strasbourg.iot-lab.info',
 'm3-5.strasbourg.iot-lab.info']

### Simulate Add
    # Stop
    # after half time
    # Start

### Simulate Remove
    # afer half time
    # Stop


In [ ]:
# Stop
if add_or_remove:
    cmd = f"iotlab-node --stop -l {site},m3,{stop_nodes}"
    run_command(cmd)

In [ ]:
# Start
if add_or_remove:
    cmd = f"iotlab-node --start -l {site},m3,{stop_nodes}"
    run_command(cmd)

In [7]:
node_site_list = [f'm3-{id_}.{site}.iot-lab.info' for id_ in nodes]
cmd = f"openv-server --iotlab-motes {' '.join(node_site_list)}"
print(cmd)

openv-server --iotlab-motes m3-1.strasbourg.iot-lab.info m3-2.strasbourg.iot-lab.info m3-5.strasbourg.iot-lab.info m3-6.strasbourg.iot-lab.info m3-9.strasbourg.iot-lab.info m3-10.strasbourg.iot-lab.info m3-19.strasbourg.iot-lab.info m3-20.strasbourg.iot-lab.info m3-21.strasbourg.iot-lab.info m3-22.strasbourg.iot-lab.info m3-23.strasbourg.iot-lab.info m3-24.strasbourg.iot-lab.info m3-29.strasbourg.iot-lab.info m3-30.strasbourg.iot-lab.info m3-33.strasbourg.iot-lab.info m3-34.strasbourg.iot-lab.info m3-37.strasbourg.iot-lab.info m3-38.strasbourg.iot-lab.info m3-47.strasbourg.iot-lab.info m3-48.strasbourg.iot-lab.info m3-49.strasbourg.iot-lab.info m3-50.strasbourg.iot-lab.info m3-51.strasbourg.iot-lab.info m3-52.strasbourg.iot-lab.info m3-53.strasbourg.iot-lab.info m3-54.strasbourg.iot-lab.info m3-57.strasbourg.iot-lab.info m3-58.strasbourg.iot-lab.info m3-61.strasbourg.iot-lab.info m3-62.strasbourg.iot-lab.info --root m3-33.strasbourg.iot-lab.info


In [63]:
root_site = f"m3-{root}.{site}.iot-lab.info"
cmd = f"openv-client root {root_site}"
print(cmd)

openv-client root m3-36.strasbourg.iot-lab.info


In [40]:
# download oml energy
exp_id = 352894
oml_path = get_log_path(folder_only=True)
cmd = f"scp -r -i ~/.ssh/id_rsa dfawwaz@{site}.iot-lab.info:/senslab/users/dfawwaz/.iot-lab/{exp_id}/consumption/ ."
print(cmd)

scp -r -i ~/.ssh/id_rsa dfawwaz@strasbourg.iot-lab.info::/senslab/users/dfawwaz/.iot-lab/352894/consumption/ .


In [43]:
cmd = f"plot_oml_consum -p -i consumption/m3-{near_root}.oml"
print(cmd)

plot_oml_consum -p -i consumption/m3-34.oml
